# Benchmark Analysis

Load pytest-benchmark JSON results from `.benchmarks/` and visualize
throughput, latency, and memory across backends.

## Prerequisites

Run benchmarks first:
```bash
docker compose -f benchmarks/infra/docker-compose.yml up -d
hatch run bench-save
```

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt

# Find the most recent benchmark JSON
bench_dir = Path("../../.benchmarks")
json_files = sorted(bench_dir.rglob("*.json"), key=lambda p: p.stat().st_mtime)
if not json_files:
    raise FileNotFoundError("No benchmark JSON files found. Run 'hatch run bench-save' first.")

latest = json_files[-1]
print(f"Loading: {latest}")
with open(latest) as f:
    data = json.load(f)

benchmarks = data["benchmarks"]
print(f"Found {len(benchmarks)} benchmark entries")

In [ ]:
# Parse benchmarks into structured records
records = []
for b in benchmarks:
    name = b["name"]
    group = b.get("group", "")
    extra = b.get("extra_info", {})
    stats = b.get("stats", {})
    params = b.get("params", {})

    # Extract backend and target from the test name
    # e.g. test_write_bytes[local-remote_store-1KB]
    parts = name.split("[")
    test_name = parts[0]
    param_str = parts[1].rstrip("]") if len(parts) > 1 else ""

    records.append(
        {
            "test": test_name,
            "params": param_str,
            "mean": stats.get("mean", 0),
            "stddev": stats.get("stddev", 0),
            "median": stats.get("median", 0),
            "rounds": stats.get("rounds", 0),
            "payload_bytes": extra.get("payload_bytes"),
            "throughput_MBps": extra.get("throughput_MBps"),
            "peak_memory_MB": extra.get("peak_memory_MB"),
        }
    )

print(f"Parsed {len(records)} records")
# Show first few
for r in records[:5]:
    print(f"  {r['test']}[{r['params']}] mean={r['mean'] * 1000:.3f}ms")

## Throughput by Backend

Compare write throughput across backends, split by target (remote-store vs raw SDK vs fsspec).

In [ ]:
# Filter write benchmarks with throughput data
write_records = [r for r in records if r["test"] == "test_write_bytes" and r["throughput_MBps"]]

if write_records:
    # Group by (backend, target)
    labels = [r["params"] for r in write_records]
    throughputs = [r["throughput_MBps"] for r in write_records]

    fig, ax = plt.subplots(figsize=(14, 6))
    bars = ax.barh(labels, throughputs)

    # Color by target type
    for bar, label in zip(bars, labels, strict=False):
        if "remote_store" in label:
            bar.set_color("#2196F3")
        elif "raw" in label or "boto3" in label or "paramiko" in label or "pathlib" in label:
            bar.set_color("#4CAF50")
        else:  # fsspec
            bar.set_color("#FF9800")

    ax.set_xlabel("Throughput (MB/s)")
    ax.set_title("Write Throughput by Backend and Target")
    ax.legend(
        handles=[
            plt.Rectangle((0, 0), 1, 1, color="#2196F3", label="remote-store"),
            plt.Rectangle((0, 0), 1, 1, color="#4CAF50", label="raw SDK"),
            plt.Rectangle((0, 0), 1, 1, color="#FF9800", label="fsspec"),
        ],
        labels=["remote-store", "raw SDK", "fsspec"],
    )
    plt.tight_layout()
    plt.show()
else:
    print("No write throughput data found. Run benchmarks with payload tests.")

## Throughput vs File Size

How does throughput scale with payload size for each backend?

In [ ]:
# Group write records by backend-target, plot throughput vs payload size
from collections import defaultdict

write_by_target = defaultdict(list)
for r in write_records:
    # Extract target label (everything before the last dash-separated size)
    parts = r["params"].rsplit("-", 1)
    if len(parts) == 2:
        target_label = parts[0]
        write_by_target[target_label].append(r)

if write_by_target:
    fig, ax = plt.subplots(figsize=(12, 6))
    for target, recs in sorted(write_by_target.items()):
        recs.sort(key=lambda r: r["payload_bytes"] or 0)
        sizes = [r["payload_bytes"] / 1024 for r in recs if r["payload_bytes"]]
        tps = [r["throughput_MBps"] for r in recs if r["payload_bytes"]]
        if sizes:
            ax.plot(sizes, tps, marker="o", label=target)

    ax.set_xlabel("Payload Size (KB)")
    ax.set_ylabel("Throughput (MB/s)")
    ax.set_title("Write Throughput vs File Size")
    ax.set_xscale("log")
    ax.legend(fontsize=8, loc="best")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No data to plot.")

## Listing Latency vs File Count

Compare listing speed at different scales.

In [ ]:
# Find listing benchmarks
list_tests = {
    "test_list_files": 50,
    "test_list_1000_files": 1000,
    "test_list_200_files_recursive": 200,
    "test_list_10k_files": 10000,
}

list_records = [r for r in records if r["test"] in list_tests]

if list_records:
    fig, ax = plt.subplots(figsize=(12, 6))

    labels = [f"{r['test']}[{r['params']}]" for r in list_records]
    means_ms = [r["mean"] * 1000 for r in list_records]

    bars = ax.barh(labels, means_ms)
    ax.set_xlabel("Latency (ms)")
    ax.set_title("Listing Latency")
    plt.tight_layout()
    plt.show()
else:
    print("No listing data found.")

## Peak Memory vs File Size

Memory usage from tracemalloc during large-file operations.

In [ ]:
# Filter records with memory data
mem_records = [r for r in records if r["peak_memory_MB"] is not None]

if mem_records:
    fig, ax = plt.subplots(figsize=(10, 6))

    payload_mb = [r["payload_bytes"] / 1_048_576 if r["payload_bytes"] else 0 for r in mem_records]
    peak_mb = [r["peak_memory_MB"] for r in mem_records]
    labels = [f"{r['test']}[{r['params']}]" for r in mem_records]

    scatter = ax.scatter(payload_mb, peak_mb, alpha=0.7, s=60)

    # Annotate points
    for x, y, label in zip(payload_mb, peak_mb, labels, strict=False):
        short = label.split("[")[-1].rstrip("]")
        ax.annotate(short, (x, y), fontsize=7, alpha=0.7)

    ax.set_xlabel("Payload Size (MB)")
    ax.set_ylabel("Peak Memory (MB)")
    ax.set_title("Peak Memory vs File Size")
    ax.grid(True, alpha=0.3)

    # Reference line: y = x (would mean entire file in memory)
    max_val = max(max(payload_mb, default=1), max(peak_mb, default=1))
    ax.plot([0, max_val], [0, max_val], "--", color="red", alpha=0.5, label="y=x (full copy)")
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No memory data found. Run large-file benchmarks.")